# Speed Up Loading The Data By Importing from the Parquet Dataset

Dataset Link here: https://www.kaggle.com/robikscube/ubiquant-parquet

Read about parquet files here: https://databricks.com/glossary/what-is-parquet

Excerpt from the above website:

**What is Parquet?**

*Parquet is an open source file format available to any project in the Hadoop ecosystem. Apache Parquet is designed for efficient as well as performant flat columnar storage format of data compared to row based files like CSV or TSV files.*

*Parquet uses the record shredding and assembly algorithm which is superior to simple flattening of nested namespaces. Parquet is optimized to work with complex data in bulk and features different ways for efficient data compression and encoding types.  This approach is best especially for those queries that need to read certain columns from a large table. Parquet can only read the needed columns therefore greatly minimizing the IO.*

Parquetとは？

Parquetは、Hadoopエコシステムのあらゆるプロジェクトで利用可能なオープンソースのファイルフォーマットです。Apache Parquetは、CSVやTSVのような行ベースのファイルと比較して、効率的でパフォーマンスの高いフラットな列指向のデータ保存形式として設計されています。

Parquetは、レコードの細断とアセンブリのアルゴリズムを使用しており、ネストした名前空間を単純に平坦化するよりも優れています。Parquetは、複雑なデータを大量に扱うために最適化されており、効率的なデータ圧縮やエンコーディングの種類を選択できるのが特徴です。このアプローチは、特に大きなテーブルから特定のカラムを読み込む必要があるクエリに最適です。Parquetは必要なカラムだけを読み込むことができるため、IOを大幅に削減することができます。

要するに、parquetの形式にすることで、効率的なデータ圧縮になる

In [1]:
import pandas as pd
import numpy as np
import gc

# Reading as CSV (Slow)
- **18GB in size**
- Don't Do this. It may cause the kaggle notebooks to crash.

In [2]:
if False:
    train = pd.read_csv('../input/ubiquant-market-prediction/train.csv')

# Reading as Parquet (Fast)
- **5.5GB** in size.
- This is faster and keeps the dtypes of the original dataset.

In [3]:
%%time
train = pd.read_parquet('../input/ubiquant-parquet/train.parquet')

FileNotFoundError: [Errno 2] No such file or directory: '../input/ubiquant-parquet/train.parquet'

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3141410 entries, 0 to 3141409
Columns: 304 entries, row_id to f_299
dtypes: float64(301), int64(2), object(1)
memory usage: 7.1+ GB


In [5]:
train.dtypes

row_id            object
time_id            int64
investment_id      int64
target           float64
f_0              float64
                  ...   
f_295            float64
f_296            float64
f_297            float64
f_298            float64
f_299            float64
Length: 304, dtype: object

In [6]:
del train
gc.collect()

118

# Reading as Parquet Low Memory (Fast & Low Mem Use)
- **3.63GB** in size
- Even better! Uses less memory and loads even faster!

In [7]:
%%time
train = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')

CPU times: user 10.3 s, sys: 13.2 s, total: 23.5 s
Wall time: 34.1 s


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3141410 entries, 0 to 3141409
Columns: 304 entries, row_id to f_299
dtypes: float32(301), object(1), uint16(2)
memory usage: 3.6+ GB


In [9]:
train.dtypes

row_id            object
time_id           uint16
investment_id     uint16
target           float32
f_0              float32
                  ...   
f_295            float32
f_296            float32
f_297            float32
f_298            float32
f_299            float32
Length: 304, dtype: object

# Read just a single `investment_id`
- If you only want to work with a single transaction load them like this

In [10]:
%%time
example = pd.read_parquet('../input/ubiquant-parquet/investment_ids/529.parquet')

CPU times: user 48.7 ms, sys: 28.6 ms, total: 77.3 ms
Wall time: 88.3 ms


In [11]:
example.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1194 entries, 315 to 3138440
Columns: 304 entries, row_id to f_299
dtypes: float64(301), int64(2), object(1)
memory usage: 2.8+ MB


# Reading a Subset of Columns

In [12]:
%%time
col_subset = ['time_id','investment_id','target','f_1','f_2','f_3']
train = pd.read_parquet('../input/ubiquant-parquet/train.parquet',
               columns=col_subset)

CPU times: user 372 ms, sys: 672 ms, total: 1.04 s
Wall time: 1.65 s


In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3141410 entries, 0 to 3141409
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   time_id        int64  
 1   investment_id  int64  
 2   target         float64
 3   f_1            float64
 4   f_2            float64
 5   f_3            float64
dtypes: float64(4), int64(2)
memory usage: 143.8 MB


## Thanks!